In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [4]:
from google.colab import files
uploaded=files.upload()

Saving users.csv to users.csv


In [11]:
#1Convert into rdd

rdd = spark.sparkContext.textFile("users.csv")

In [19]:
#df=rdd.toDF()
#df = spark.createDataFrame(rdd)
df = rdd.map(lambda x: x.split("::")).toDF()

In [23]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema=StructType([\
                  StructField("S_no",IntegerType(),True),\
                  StructField("Gender",StringType(),True),\
                  StructField("Movie_id",IntegerType(),True),\
                  StructField("rating",IntegerType(),True),\
                  StructField("watch_count",IntegerType(),True)\
                   ])

In [32]:
df=spark.read.csv("users.csv",sep="::",schema=schema,header=True,inferSchema=True)
df.show()

+----+------+--------+------+-----------+
|S_no|Gender|Movie_id|rating|watch_count|
+----+------+--------+------+-----------+
|null|  null|    null|  null|       null|
|null|  null|    null|  null|       null|
|   1|     F|       1|    10|      48067|
|   2|     M|      56|    16|      70072|
|   3|     M|      25|    15|      55117|
|   4|     M|      45|     7|       2460|
|   5|     M|      25|    20|      55455|
|   6|     F|      50|     9|      55117|
|   7|     M|      35|     1|       6810|
|   8|     M|      25|    12|      11413|
|   9|     M|      25|    17|      61614|
|  10|     F|      35|     1|      95370|
|  11|     F|      25|     1|       4093|
|  12|     M|      25|    12|      32793|
|  13|     M|      45|     1|      93304|
|  14|     M|      35|     0|      60126|
|  15|     M|      25|     7|      22903|
|  16|     F|      35|     0|      20670|
|  17|     M|      50|     1|      95350|
|  18|     F|      18|     3|      95825|
+----+------+--------+------+-----

In [35]:
df1=df.na.drop()
df1.show()

+----+------+--------+------+-----------+
|S_no|Gender|Movie_id|rating|watch_count|
+----+------+--------+------+-----------+
|   1|     F|       1|    10|      48067|
|   2|     M|      56|    16|      70072|
|   3|     M|      25|    15|      55117|
|   4|     M|      45|     7|       2460|
|   5|     M|      25|    20|      55455|
|   6|     F|      50|     9|      55117|
|   7|     M|      35|     1|       6810|
|   8|     M|      25|    12|      11413|
|   9|     M|      25|    17|      61614|
|  10|     F|      35|     1|      95370|
|  11|     F|      25|     1|       4093|
|  12|     M|      25|    12|      32793|
|  13|     M|      45|     1|      93304|
|  14|     M|      35|     0|      60126|
|  15|     M|      25|     7|      22903|
|  16|     F|      35|     0|      20670|
|  17|     M|      50|     1|      95350|
|  18|     F|      18|     3|      95825|
|  19|     M|       1|    10|      48073|
|  20|     M|      25|    14|      55113|
+----+------+--------+------+-----

In [45]:
df1.dropDuplicates(['S_no','Gender','Movie_id','rating','watch_count']).show()
#df1.dropDuplicates().show()
#df2=df1.select(['S_no','Gender','Movie_id','rating','watch_count']).distinct().collect()

+----+------+--------+------+-----------+
|S_no|Gender|Movie_id|rating|watch_count|
+----+------+--------+------+-----------+
| 191|     M|      18|     4|       4915|
| 315|     F|      56|     1|      55105|
| 646|     M|      35|     7|       8505|
| 743|     F|       1|     2|      60660|
| 881|     M|      18|    14|      76401|
|1011|     M|      25|     8|      92115|
|1015|     M|      35|     3|      11220|
|1061|     M|      45|     0|      46060|
|1215|     M|      35|    17|      23602|
|1305|     M|      45|    20|      94121|
|1392|     M|      45|     3|      90006|
|1745|     M|      45|     0|      94114|
|1949|     M|      18|    17|      92595|
|2501|     M|      25|    15|      37923|
|2587|     M|      35|     7|      55124|
|2730|     M|      35|    11|      48640|
|2796|     M|      25|    14|      92104|
|3067|     F|      25|     0|       2148|
|3221|     M|      25|     0|      19106|
|3482|     M|      50|    17|       8043|
+----+------+--------+------+-----

In [66]:
df1.select('S_no','Gender','Movie_id','rating','watch_count').dropDuplicates().show()

+----+------+--------+------+-----------+
|S_no|Gender|Movie_id|rating|watch_count|
+----+------+--------+------+-----------+
| 191|     M|      18|     4|       4915|
| 315|     F|      56|     1|      55105|
| 646|     M|      35|     7|       8505|
| 743|     F|       1|     2|      60660|
| 881|     M|      18|    14|      76401|
|1011|     M|      25|     8|      92115|
|1015|     M|      35|     3|      11220|
|1061|     M|      45|     0|      46060|
|1215|     M|      35|    17|      23602|
|1305|     M|      45|    20|      94121|
|1392|     M|      45|     3|      90006|
|1745|     M|      45|     0|      94114|
|1949|     M|      18|    17|      92595|
|2501|     M|      25|    15|      37923|
|2587|     M|      35|     7|      55124|
|2730|     M|      35|    11|      48640|
|2796|     M|      25|    14|      92104|
|3067|     F|      25|     0|       2148|
|3221|     M|      25|     0|      19106|
|3482|     M|      50|    17|       8043|
+----+------+--------+------+-----

In [57]:
#groupby to the movie_id
df1.groupBy("Movie_id").count()

Movie_id,count
1,220
35,1184
50,485
45,542
25,2078
56,371
18,1094


In [59]:
#df.groupby(["Movie_id"]).transform("count")
#df.groupby(["Movie_id"])["id"].count().reset_index(name="count")
#import pandas as pd
#df['count'] = df.groupby('Movie_id')['Movie_id'].transform('count')
#df1['count'] = df1.groupBy('Movie_id')['Movie_id'].transform('count')
#df1.show()
#count=df1.groupBy('Movie_id').count()
#count.show()
#import pyspark.sql.functions as f
#df.groupBy('Movie_id').count().select('Movie_id', f.col('count').alias('count')).show()

+--------+-----+
|Movie_id|count|
+--------+-----+
|    null|    2|
|       1|  222|
|      35| 1193|
|      50|  496|
|      45|  550|
|      25| 2096|
|      56|  380|
|      18| 1103|
+--------+-----+



In [60]:
#2 Convert the dataframe into rdd

rdd = df.rdd

In [65]:
#3 lit() method:

#The function lit() is used to add a new column to the dataframe by assigning a literal or constant value to it.

#Eg:
from pyspark.sql.functions import (array, col, create_map, lit, when)
final = df1.select(col("Gender"),lit("22").alias("lit_col")).show()

#here i gave a constant value 22 for Gender

+------+-------+
|Gender|lit_col|
+------+-------+
|     F|     22|
|     M|     22|
|     M|     22|
|     M|     22|
|     M|     22|
|     F|     22|
|     M|     22|
|     M|     22|
|     M|     22|
|     F|     22|
|     F|     22|
|     M|     22|
|     M|     22|
|     M|     22|
|     M|     22|
|     F|     22|
|     M|     22|
|     F|     22|
|     M|     22|
|     M|     22|
+------+-------+
only showing top 20 rows



In [ ]:
#4 diff bw groupBy and partitionBy

'''groupBy is more a logical way to group your data. It looks like a groupBy, it normally reduces the number of rows returned by calculating averages or sums for each row
   PartitionBy is more physical, you really physically partition your data in the cluster, it is used to partition the large dataset into smaller files based on one or multiple columns.'''

In [68]:
#5 Write an sql statement on the existing dataframe with an condition to fetch the maximum count_no

df1.createOrReplaceTempView("ME")
spark.sql("SELECT COUNT(*) FROM ME").show()
#for Movie_id column
spark.sql("SELECT COUNT(distinct Movie_id) FROM ME").show()

+--------+
|count(1)|
+--------+
|    5974|
+--------+

+------------------------+
|count(DISTINCT Movie_id)|
+------------------------+
|                       7|
+------------------------+



In [70]:
#6 Create a temporary table from the Existing dataframe
from pyspark.sql import SQLContext
df1.registerTempTable('PritamTable')
sqlContext = SQLContext(spark)

In [86]:
#6.1 Self join on temp table

self_join= spark.sql("SELECT * FROM PritamTable e LEFT OUTER JOIN PritamTable d ON e.S_no == d.watch_count") \
.show(truncate=False)

+----+------+--------+------+-----------+----+------+--------+------+-----------+
|S_no|Gender|Movie_id|rating|watch_count|S_no|Gender|Movie_id|rating|watch_count|
+----+------+--------+------+-----------+----+------+--------+------+-----------+
|1   |F     |1       |10    |48067      |null|null  |null    |null  |null       |
|2   |M     |56      |16    |70072      |null|null  |null    |null  |null       |
|3   |M     |25      |15    |55117      |null|null  |null    |null  |null       |
|4   |M     |45      |7     |2460       |null|null  |null    |null  |null       |
|5   |M     |25      |20    |55455      |null|null  |null    |null  |null       |
|6   |F     |50      |9     |55117      |null|null  |null    |null  |null       |
|7   |M     |35      |1     |6810       |null|null  |null    |null  |null       |
|8   |M     |25      |12    |11413      |null|null  |null    |null  |null       |
|9   |M     |25      |17    |61614      |null|null  |null    |null  |null       |
|10  |F     |35 

In [87]:
#6.2 union operation on the same table

spark.sql("select * from PritamTable union select * from PritamTable").show()

+----+------+--------+------+-----------+
|S_no|Gender|Movie_id|rating|watch_count|
+----+------+--------+------+-----------+
| 100|     M|      35|    17|      95401|
| 497|     F|      25|    17|      55412|
| 734|     M|      25|     7|      90078|
| 753|     M|       1|    10|      42754|
| 783|     M|      25|     4|      53703|
| 830|     M|      35|     7|      60657|
| 944|     M|      35|     0|      91505|
| 952|     M|      56|    16|      97478|
|1073|     F|      25|     6|      72211|
|1143|     M|      50|     7|      94024|
|1767|     M|      25|    20|      75205|
|1907|     M|      35|    17|      94804|
|1908|     M|      56|    13|      95129|
|2138|     F|      18|     4|      88119|
|2174|     M|      50|    12|      87505|
|2247|     M|      45|     2|      60613|
|2293|     F|      56|     3|      48130|
|2323|     M|      35|     3|      13088|
|2688|     M|      25|    17|      54915|
|3093|     F|      45|    20|      89134|
+----+------+--------+------+-----

In [88]:
# delete the temp table

spark.catalog.dropTempView("PritamTable")

In [89]:
#7
#PySpark dataframe can be converted to pandas dataframe using the function toPandas() method
pandasdf = df1.toPandas()
print(pandasdf)

      S_no Gender  Movie_id  rating  watch_count
0        1      F         1      10        48067
1        2      M        56      16        70072
2        3      M        25      15        55117
3        4      M        45       7         2460
4        5      M        25      20        55455
...    ...    ...       ...     ...          ...
5969  6036      F        25      15        32603
5970  6037      F        45       1        76006
5971  6038      F        56       1        14706
5972  6039      F        45       0         1060
5973  6040      M        25       6        11106

[5974 rows x 5 columns]


In [90]:
#8 
data = [("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"), \
("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"), \
("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"), \
("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico")]
columns= ["Product","Amount","Country"]
df=spark.createDataFrame(data=data,schema=columns)

In [91]:
df.show()

+-------+------+-------+
|Product|Amount|Country|
+-------+------+-------+
| Banana|  1000|    USA|
|Carrots|  1500|    USA|
|  Beans|  1600|    USA|
| Orange|  2000|    USA|
| Orange|  2000|    USA|
| Banana|   400|  China|
|Carrots|  1200|  China|
|  Beans|  1500|  China|
| Orange|  4000|  China|
| Banana|  2000| Canada|
|Carrots|  2000| Canada|
|  Beans|  2000| Mexico|
+-------+------+-------+

